In [2]:
from bokeh.plotting import figure, show, output_file, ColumnDataSource
from bokeh.sampledata.us_states import data as states
from bokeh.models import HoverTool
from data_classes import unaltered, census
from areas_by_region import get_states_abb_into_region

us_states = states.copy()
del us_states["HI"]
del us_states["AK"]
del us_states["DC"]

state_xs = [states[code]["lons"] for code in us_states]
state_ys = [states[code]["lats"] for code in us_states]


regions = ['South', 'Northeast', 'Midwest', 'West']
region_names = []
grouped_states = {}

bls_colors = ["#400000", "#B00000", "#FF0000", "#680000"]

bls_region_colors = []
region_bls_stats_2015 = []
region_bls_stats_2016 = []

census_colors = ["#C76E06", "#FFA000", "#FC8900", "#FF6000"]

census_region_colors = []
region_census_stats_2015 = []
region_census_stats_2016 = []

for region in regions:
    abb = get_states_abb_into_region(region)
    grouped_states[region] = abb
    
for name in us_states:
    for key, item in grouped_states.items():
        if name not in ['DC', 'HI', 'AK']:
            if name in item:
                try:
                    region_names.append(key)
                    bls_region_colors.append(bls_colors[regions.index(key)])
                    census_region_colors.append(census_colors[regions.index(key)])
                    for stat in unaltered:
                        if stat[0] == key:
                            region_bls_stats_2015.append([stat[1]])
                            region_bls_stats_2016.append([stat[2]])
                    for info in census:
                        if info[0] == key:
                            region_census_stats_2015.append([info[1]])
                            region_census_stats_2016.append([info[2]])
                except KeyError:
                    bls_region_colors.append("white")
                
bls_source = ColumnDataSource(data=dict(
    x = state_xs,
    y = state_ys,
    color = bls_region_colors,
    name = region_names,
    nojob2015 = region_bls_stats_2015,
    nojob2016 = region_bls_stats_2016
))

census_source = ColumnDataSource(data=dict(
    x = state_xs,
    y = state_ys,
    color = census_region_colors,
    name = region_names,
    nojob2015 = region_census_stats_2015,
    nojob2016 = region_census_stats_2016
))

TOOLS="pan,wheel_zoom,box_zoom,reset,hover,save"

p = figure(title="U.S. Bureau of Labor Statistics Unemployed No Job Available", toolbar_location="right",
           plot_width=1100, plot_height=650, tools=TOOLS)

p.patches('x', 'y', source=bls_source,
          fill_color='color', fill_alpha=0.7,
          line_color="#B0A6A4", line_width=0.5)

phover = p.select_one(HoverTool)
phover.tooltips = [
    ("Name", "@name"),
    ("Population", "With No Job Available"),
    ("2015", "@nojob2015"),
    ("2016", "@nojob2016"),
]

h = output_file("BLS No Job Awaits.html", title="BLSPopWNoJobs.py")

show(p)

q = figure(title="U.S. Census + Bureau of Labor Statistics Unemployed No Job Available", toolbar_location="right",
           plot_width=1100, plot_height=650, tools=TOOLS)

q.patches('x', 'y', source=census_source,
          fill_color='color', fill_alpha=0.7,
          line_color="#FFFFFF", line_width=0.5)

qhover = q.select_one(HoverTool)
qhover.tooltips = [
    ("Name", "@name"),
    ("Population", "With No Job Available"),
    ("2015", "@nojob2015"),
    ("2016", "@nojob2016"),
]

j = output_file("Census No Job Awaits.html", title="CensusBLSPopWNoJobs.py")

show(q)